# Data Engineering Task - Pevex

In this Python script I scraped data from a website "pevex.hr". 

The code defines several empty lists to store data.

The code then defines a function called "get_base_url" that takes a category argument and returns the base URL for that category.

The code also defines a dictionary called "category_ranges" that contains the start and end pages for each category.

The code then loops through each category and page range, retrieves the HTML content for each page, and uses BeautifulSoup to extract the relevant data from the HTML.

For each product, the code extracts the product category, product code, and product ID.

The code then extracts the product title, which includes the brand and model, and separates the model from the rest of the title. It also extracts the product description URL and retrieves the HTML content for that URL.

The code then uses BeautifulSoup to extract more information about each product, such as weight, height, width, length, warranty, brand, color, and power.

Finally, the code extracts the euro and kuna prices for each product.

The extracted data is then added to the corresponding lists. Once all the data has been extracted, it is assembled into a Pandas DataFrame with columns for each type of data.

In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

product_cats=[]
product_codes=[]
product_ids=[]
brands = []
models = []
opis_modela = []
euro_prices = []
kuna_prices = []
weights = []
warranties = []
heights = []
widths = []
lengths = []
colors =[]
powers=[]

def get_base_url(category):
    return f"https://pevex.hr/kucanski-uredjaji/{category}?page_size=20&sort=3&page_number="

categories = ['mali-kucanski-aparati-kucanstvo', 'bijela-tehnika', 'hladjenje-i-grijanje', 'mali-kucanski-aparati-osobna-njega',
             'mali-kucanski-aparati-priprema-hrane']
category_ranges = {
    "mali-kucanski-aparati-kucanstvo": (1, 17),
    "bijela-tehnika": (1, 40),
    "hladjenje-i-grijanje":(1, 11),
    "mali-kucanski-aparati-osobna-njega": (1, 8),
    "mali-kucanski-aparati-priprema-hrane": (1, 20)
    
}

for category in categories:
    baseURL = get_base_url(category)
    start, end = category_ranges[category]
    for i in range(start, end+1):
        base_url = f"{baseURL}{i}"
        
        response = requests.get(base_url)
        
        soup = BeautifulSoup(response.content, 'html.parser')
    
        os_info=soup.find_all('div', class_='product-display-grid item')
        for info in os_info:
            product_cat=info['data-product-categories']
            product_cats.append(product_cat)
        
            product_code=info['data-product-code']
            product_codes.append(product_code)
        
            product_id=info['data-product-id']
            product_ids.append(product_id)
    
        product = soup.find_all('a', class_='product-click')

        for p in product:
            title = p['title']
            nemodel, _, model = title.partition(' ')
            models.append(model)
            opis_modela.append(p['href'])
        
            detail_url = "https://pevex.hr"+p['href']
            detail_response = requests.get(detail_url)
            detail_soup = BeautifulSoup(detail_response.content, 'html.parser')

            detail_info = detail_soup.find_all('div', class_='item')
            weight = None
            height = None
            width = None
            length = None
            warranty= None
            brand=None
            color=None
            power=None
        
            for detail in detail_info:
                # get the left and right divs
                left_div = detail.find('div', class_='left')
                right_div = detail.find('div', class_='right')
                # get the text content from the left and right divs
        
                if left_div is not None:
                    left_text = left_div.get_text().strip()
                else:
                    left_text = None

                if right_div is not None:
                    right_text = right_div.find('div', class_='width-100').get_text().strip()
                else:
                    right_text = None

                # append the right text to the corresponding list based on the left text
                if left_text == 'Jamstvo':
                    warranty=right_text
                if left_text == 'Težina':
                    weight = right_text
                if left_text == 'Visina':
                    height = right_text
                if left_text == 'Širina':
                    width = right_text
                if left_text == 'Dužina':
                    length = right_text
                if left_text == 'Robna marka':
                    brand = right_text
                if left_text == 'Boja':
                    color = right_text
                if left_text == 'Snaga (W)':
                    power = right_text
        
            weights.append(weight)
            warranties.append(warranty)
            heights.append(height)
            widths.append(width)
            lengths.append(length)
            brands.append(brand)
            colors.append(color)
            powers.append(power)
            
        prices = soup.find_all('div', class_='actions-wrapper')

        for price in prices:
            if ('pk-special' in price['class']) or ('discount' in price['class']) or ('price pk pk-special' in price['class']):
                continue
            if price.find('div', class_='price'):
                price_values = price.text.split('/')
                euro_price_raw = price_values[0].strip().replace('\n\n\n\n\n\n\n\nDodaj u košaricu', '')
                euro_price= euro_price_raw.replace('.', '').replace(',', '.')
                kuna_price_raw = price_values[1].strip().replace('\n\n\n\n\n\n\n\nDodaj u košaricu', '')
                kuna_price = kuna_price_raw.split('\n')[0].replace('.', '').replace(',', '.') if kuna_price_raw else None
                euro_prices.append(euro_price)
                kuna_prices.append(kuna_price)
            else:
                euro_prices.append(None)
                kuna_prices.append(None)


df = pd.DataFrame({'Code':product_codes, 'ProductID':product_ids,'Category':product_cats,'Brand':brands,'Model': models, 'Opis_modela': opis_modela,'Warrenty':warranties, 'Color':colors,'Power':powers, 'Weight': weights, 'Height': heights, 'Width': widths, 'Length': lengths, 'Euro Price':euro_prices, 'Kuna Price': kuna_prices})
display(df)

,Code,ProductID,Category,Brand,Model,Opis_modela,Warrenty,Color,Power,Weight,Height,Width,Length,Euro Price,Kuna Price
0,375585,76118,/Kućanski uređaji/Mali kućanski aparati - Kuća...,XIAOMI,E10 EU,/robotski-usisavac-xiaomi-e10-eu,24 mj.,Bijela,None,"3,20 kg",8 cm,33 cm,33 cm,193.64 €,1458.98 kn
1,375586,76119,/Kućanski uređaji/Mali kućanski aparati - Kuća...,XIAOMI,S10,/robotski-usisavac-xiaomi-s10,24 mj.,Bijela,None,"4,77 kg",9 cm,35 cm,35 cm,283.39 €,2135.20 kn
2,375561,76104,/Kućanski uređaji/Mali kućanski aparati - Kuća...,HISENSE,HVC6264BK,/stapni-usisavac-hisense-hvc6264bk,24 mj.,Siva,None,"3,00 kg",111 cm,25 cm,25 cm,225.50 €,1699.03 kn
3,375564,76107,/Kućanski uređaji/Mali kućanski aparati - Kuća...,HISENSE,RVCL144AB,/robotski-usisavac-hisense-rvcl144ab,24 mj.,Crna/Siva,None,"3,40 kg",10 cm,35 cm,35 cm,490.90 €,3698.69 kn
4,375565,76108,/Kućanski uređaji/Mali kućanski aparati - Kuća...,HISENSE,RVCLS144B,/robotski-usisavac-hisense-rvcls144b,24 mj.,Crna/Siva,None,"9,00 kg",10 cm,35 cm,35 cm,796.20 €,5998.97 kn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1892,308366,31514,/Kućanski uređaji/Mali kućanski aparati - Prip...,STATUS,SV360 PROVAC,/aparat-za-vakumiranje-status-sv360-provac,60 mj.,None,None,"8,96 kg",14 cm,28 cm,51 cm,None,None
1893,359283,62763,/Kućanski uređaji/Mali kućanski aparati - Prip...,SENCOR,SSJ 4042RD SPORI,/sokovnik-sencor-ssj-4042rd-spori,24 mj.,None,None,"4,77 kg",41 cm,26 cm,35 cm,None,None
1894,332452,44609,/Kućanski uređaji/Mali kućanski aparati - Prip...,GORENJE,R707A,/mesoreznica-gorenje-r707a,24 mj.,None,None,"2,90 kg",23 cm,36 cm,26 cm,None,None
1895,335118,46737,/Kućanski uređaji/Mali kućanski aparati - Prip...,SENCOR,STM4467CH 1000W,/multipraktik-sencor-stm4467ch-1000w,24 mj.,None,None,"8,48 kg",50 cm,30 cm,60 cm,None,None


In [45]:
df.to_csv('kućanskiaparati2.csv', index=False)

In [54]:
unique_count = df['ProductID'].nunique()

print(unique_count)

1896


In [56]:
non_unique = df[df['ProductID'].duplicated(keep=False)]

print(non_unique)

        Code ProductID                          Category  Brand        Model  \
409   369238     71634  /Kućanski uređaji/Bijela tehnika  CANDY  FIDC B605 L   
1017  369238     71634  /Kućanski uređaji/Bijela tehnika  CANDY  FIDC B605 L   

                               Opis_modela Warrenty   Color Power    Weight  \
409   /ugradbena-pecnica-candy-fidc-b605-l   24 mj.  Bijela  None  27,40 kg   
1017  /ugradbena-pecnica-candy-fidc-b605-l   24 mj.  Bijela  None  27,40 kg   

     Height  Width Length Euro Price  Kuna Price  
409   60 cm  60 cm  57 cm   318.40 €  2398.98 kn  
1017  60 cm  60 cm  57 cm       None        None  


In [57]:
df = df.drop([1017])

I had to drop one row because it was duplicate

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1896 entries, 0 to 1896
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Code         1896 non-null   object
 1   ProductID    1896 non-null   object
 2   Category     1896 non-null   object
 3   Brand        1837 non-null   object
 4   Model        1896 non-null   object
 5   Opis_modela  1896 non-null   object
 6   Warrenty     1702 non-null   object
 7   Color        966 non-null    object
 8   Power        291 non-null    object
 9   Weight       1896 non-null   object
 10  Height       1846 non-null   object
 11  Width        1851 non-null   object
 12  Length       1844 non-null   object
 13  Euro Price   1674 non-null   object
 14  Kuna Price   1674 non-null   object
dtypes: object(15)
memory usage: 237.0+ KB


This code connects to a PostgreSQL database and creates a table named "Kucanski_aparati2" with 13 columns, including product_id, code, category, brand, model, opis_modela, warranty, weight, height, width, length, euro_price, and kuna_price. The data for this table is being taken from a pandas dataframe named "df". The code then loops through each row of the dataframe and inserts the data into the table using an SQL insert statement. Finally, the code commits the changes to the database and closes the connection.

Table "Kucanski_aparati2" has 13 instead original 15 columns beacuse I didn't include columns 'color' and 'power' beacuse there is not enaugh data in those columns.

In [58]:

import psycopg2

# Connect to the database
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="pevex",
    user="postgres",
    password="123456789"
)

# Create a cursor
cur = conn.cursor()

# Create the table
cur.execute('''
    CREATE TABLE IF NOT EXISTS Kucanski_aparati2 (
        product_id VARCHAR(255) PRIMARY KEY,
        code VARCHAR(255),
        category VARCHAR(255),
        brand VARCHAR(255),
        model VARCHAR(255),
        opis_modela VARCHAR(255),
        warranty VARCHAR(255),
        weight VARCHAR(255),
        height VARCHAR(255),
        width VARCHAR(255),
        length VARCHAR(255),
        euro_price VARCHAR(255),
        kuna_price VARCHAR(255)
    )
''')

# Insert the data into the table
for i, row in df.iterrows():
    cur.execute('''
        INSERT INTO Kucanski_aparati2 (
            category, code, product_id, brand, model, opis_modela, warranty,
            weight, height, width, length, euro_price, kuna_price
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
    ''', (
        row['Category'], row['Code'], row['ProductID'], row['Brand'], row['Model'],
        row['Opis_modela'], row['Warrenty'], row['Weight'], row['Height'], row['Width'],
        row['Length'], row['Euro Price'], row['Kuna Price']
    ))

# Commit the changes and close the connection
conn.commit()
cur.close()
conn.close()

In the next phase of the task, I will use the dataset of kućanski_aparati 1 and not specifically the one that is the result of this scraping described above. The only reason for that is the first dataset contains 2168 rows. That data set is the result of scraping done approx. 10 days earlier. In those 10 days, pevex removed some products from the site, and when I did the second scraping, there were fewer products. The difference in the code is that in the first scaping, the code did not include the function and the first for loop that goes through all pages of the category,so the same code was run 5 times for each category. In the meantime, I have upgraded the code and this is its final version described above.